## $\underline{\textbf{SQL Notes}}$

<h1 align="center">Queries</h1> 
statement declaring what data to look for, where in database, and optionally how to transform it before it's returned.

$\underline{\text{simple select query for specific columns}}:$  
``` sql
SELECT column_name, another_column, …
FROM mytable;
```
#can use asteric (*) to indicate all.

$\underline{\text{select query w/ constraint}}:$
``` sql
SELECT column_name, another_column, …
FROM mytable
WHERE condition
    AND/OR another_condition
    AND/OR ..;
```
might find these operators useful: 

<div class="datatable">
    <table class="table table-striped table-condensed">
        <tbody><tr>
            <td style="width: 20%; text-align: center;">Operator</td>
            <td style="width: 50%">Condition</td>
            <td>SQL Example</td>
        </tr>
        <tr>
            <td style="text-align: center;">=, !=, &lt; &lt;=, &gt;, &gt;=</td>
            <td>Standard numerical operators</td>
            <td>col_name <span class="faux-keyword">!=</span> 4</td>
        </tr>
        <tr>
            <td style="text-align: center;">BETWEEN … AND …</td>
            <td>Number is within range of two values (inclusive)</td>
            <td>col_name <span class="faux-keyword">BETWEEN</span> 1.5 <span class="faux-keyword">AND</span> 10.5</td>
        </tr>
        <tr>
            <td style="text-align: center;">NOT BETWEEN … AND …</td>
            <td>Number is not within range of two values (inclusive)</td>
            <td>col_name <span class="faux-keyword">NOT BETWEEN</span> 1 <span class="faux-keyword">AND</span> 10</td>
        </tr>
        <tr>
            <td style="text-align: center;">IN (…)</td>
            <td>Number exists in a list</td>
            <td>col_name <span class="faux-keyword">IN</span> (2, 4, 6)</td>
        </tr>
        <tr>
            <td style="text-align: center;">NOT IN (…)</td>
            <td>Number does not exist in a list</td>
            <td>col_name <span class="faux-keyword">NOT IN</span> (1, 3, 5)</td>
        </tr>
    </tbody></table>
</div>

expanded operator table:
<div class="datatable">
    <table class="table table-striped table-condensed">
        <tbody><tr>
            <td style="width: 15%; text-align: center;">Operator</td>
            <td style="width: 60%">Condition</td>
            <td>Example</td>
        </tr>
        <tr>
            <td style="text-align: center;">=</td>
            <td>Case sensitive exact string comparison (<em>notice the single equals</em>)</td>
            <td>col_name <span class="faux-keyword">=</span> "abc"</td>
        </tr>
        <tr>
            <td style="text-align: center;">!= or &lt;&gt;</td>
            <td>Case sensitive exact string inequality comparison</td>
            <td>col_name <span class="faux-keyword">!=</span> "abcd"</td>
        </tr>
        <tr>
            <td style="text-align: center;">LIKE</td>
            <td>Case insensitive exact string comparison</td>
            <td>col_name <span class="faux-keyword">LIKE</span> "ABC"</td>
        </tr>
        <tr>
            <td style="text-align: center;">NOT LIKE</td>
            <td>Case insensitive exact string inequality comparison</td>
            <td>col_name <span class="faux-keyword">NOT LIKE</span> "ABCD"</td>
        </tr>
        <tr>
            <td style="text-align: center;">%</td>
            <td>Used anywhere in a string to match 
                a sequence of zero or more characters (only with LIKE or NOT LIKE)</td>
            <td>col_name <span class="faux-keyword">LIKE</span> "%AT%"<br>
                (matches "<span class="uline">AT</span>", "<span class="uline">AT</span>TIC", "C<span class="uline">AT</span>" 
                    or even "B<span class="uline">AT</span>S")</td>
        </tr>
        <tr>
            <td style="text-align: center;">_</td>
            <td>Used anywhere in a string to match 
                a single character (only with LIKE or NOT LIKE)</td>
            <td>col_name <span class="faux-keyword">LIKE</span> "AN_"<br>
                (matches "<span class="uline">AN</span>D", but not "<span class="uline">AN</span>")</td>
        </tr>
        <tr>
            <td style="text-align: center;">IN (…)</td>
            <td>String exists in a list</td>
            <td>col_name <span class="faux-keyword">IN</span> ("A", "B", "C")</td>
        </tr>
        <tr>
            <td style="text-align: center;">NOT IN (…)</td>
            <td>String does not exist in a list</td>
            <td>col_name <span class="faux-keyword">NOT IN</span> ("D", "E", "F")</td>
        </tr>
    </tbody></table>
</div>

notes that full-text search is better left to deticated libraries like $\textbf{Apache Lucene}$ or $\textbf{Sphinx}$.

$\textbf{Ex}.$ to find all Toy Story movies:
``` sql
SELECT *
FROM movies
WHERE Title LIKE "%Toy Story%";
```

<h1 align="center">Filtering</h1> 

$\textbf{DISTINCT}:$ keyword that'll blindly discard duplicated values when querrying
``` sql
SELECT DISTINCT column1, column2, ...
FROM mytable
WHERE conditions;
```
Since the $\text{DISTINCT}$ keyword will blindly remove duplicate rows, we will learn in a future lesson how to discard duplicates based on specific columns using grouping and the $\text{GROUP BY}$ clause.

$\textbf{ORDER BY}:$ clause to sort results by a given column in ASCending or DESCending order.
``` sql
SELECT column1, column2, ...
FROM mytable
WHERE conditions
ORDER BY column1 ASC/DESC;
```
each row is sorted alpha-numerically based on the specified column's values, in some databases you can also specify a collation to better soty data containing multi-language text.

$\textbf{LIMIT and OFFSET}:$ clauses commonly used with $\text{ORDER BY}$. the $\text{LIMIT}$ will reduce the number of rows to return, and the optional $\text{OFFSET}$ will specify where to begin counting the number rows from.
``` sql
SELECT column1, column2, ...
FROM mytable
WHERE conditions
ORDER BY column1 ASC/DESC
LIMIT num_limit OFFSET num_offset;
```

$\textbf{Ex.}$ to list pixar movies sorted alphabetically offseted by 5:
``` sql
SELECT title FROM movies
ORDER BY title ASC
LIMIT 5 OFFSET 5;
```

<h1 align="center">Database Normalization</h1>  

going from single table -> multiple orthogonal tables, using normalization.

main purpose: removing duplicate entries while allowing the different tables to change independent of eachother. 

queries get more complex since they have to find data from different parts of the databases. there are thus also performence issues.


$\textbf{JOINs}:$  
tables that share info about a single entity need to have a $\it{primary \space key}$ that identifies that entity $\it{uniquely}$ across the database.

common primary key type is an auto-incrementing integer (space efficient), but it can be any unique key, such as string, hashed value, etc.

the $\text{JOIN}$ clause in a query can combine row data across two seperate tables using a unique key.

$\textbf{INNER JOIN}:$  
a process that matches rows from the first table and the second which have the same key (as defined by the $\text{ON}$ constraint). 

``` sql
SELECT column1, column2, ...
FROM mytable
INNER JOIN table2
    ON mytable.id = table2.id
WHERE conditions
ORDER BY column, ... ASC/DESC
LIMIT num_limit OFFSET num_offest;
```

this results in a row with combined columns from both tables. after the tables are joined, the other clauses are applied.

NOTE: $\text{INNER JOIN}$ can be written as $\text{JOIN}$, however we use the first to dinstinguish from other types of joins. 

$\textbf{Ex.}$ two tables, "Movies" and "Boxoffice" that are to be joined, want to show movies where int sales greater than domestic, ordered by rating desc:

``` sql
SELECT * FROM Movies
INNER JOIN Boxoffice
    ON Movies.Id = Boxoffice.Movie_id
WHERE International_sales > Domestic_sales
ORDER BY Rating DESC;
```

$\textbf{OUTER JOIN}:$